In [1]:
import numpy as np
import pymysql
import matplotlib.pyplot as plt
import astropy.io.fits as fits
import pandas as pd

# Generating a Convenient Science-Ready Dataframe


In this notebook, I compile all the data being collected from various sources into one dataframe with which we can conduct our science. The data involved is as follows:

    1. SDSS x ALMA matched target list from xmatch_Oct2024.ipynb
    2. Gas masses from salvage-post-processing.ipynb
    3. Visual classifications of the images and spectra
    4. Basic SDSS parameters from local UVic SDSS catalogues
    5. Line flux values from local UVic SDSS catalogues
    6. WISE magnitudes and AGN flag from local UVic SDSS catalogues
    7. LERG/HERG flags from Best et al. catalogue
    8. SDSS Sersic fits from Simard et al. catalogue
    9. SDSS photometric fits from SDSS DR7 modelmag pipeline
    10. Gas masses from physical apertures imposed on ALMA cubes
    11. PSB flags from Wilkinson et al. (2022)
    12. TO ADD: final resolution from headers of the cubes

## 1. Load in the SALVAGE target list

In [2]:
fpath = '/arc/projects/salvage/ALMA_reduction/samples/'
file  = 'salvage-Oct16-sample_match-lt-4rp_mrs-gt-1p5rp_res-lt-3p05_qa2-pass_no-inf-dups.txt'

objID_sample, year_sample, muid_sample, auid_sample, proj_sample, name_sample = np.loadtxt(fpath+file, unpack = True, dtype = str, usecols = [0,9,10,11,12,13])
z_sample, mass_sample, rpetro_sample, ra_sample, dec_sample, res_sample, mrs_sample, sens_sample  = np.loadtxt(fpath+file, unpack = True, dtype = float, usecols = [1,2,3,4,5,6,7,8])

In [3]:
d = {'objID': objID_sample, 'source_name': name_sample, 'project_code': proj_sample, 'year': year_sample, 'MUID': muid_sample, 'AUID': auid_sample, \
    'ra': ra_sample, 'dec': dec_sample, 'z': z_sample, 'rpetro': rpetro_sample, 'MRS': mrs_sample, 'resolution': res_sample, 'sensitivity': sens_sample}
df = pd.DataFrame(data=d)
df

,objID,source_name,project_code,year,MUID,AUID,ra,dec,z,rpetro,MRS,resolution,sensitivity
0,587726033335550094,Abell,2011.0.00374.S,2011,uid://A002/X327408/X22f,uid://A002/X3b3400/X64c,210.258631,2.878469,0.251876,2.91803,10.534628,1.333456,1.500086
1,587741490891325518,587741490891325518_8241-3704,2018.1.00541.S,2018,uid://A001/X133d/Xe83,uid://A002/Xd74c3f/X24ad,126.568894,17.362436,0.065923,4.65495,21.589244,1.802884,0.914182
2,587722984441250023,HATLAS_J143336.7+005042,2017.1.00025.S,2017,uid://A001/X1284/X1534,uid://A002/Xca795f/X25d1,218.403317,0.845238,0.069836,1.43384,14.286183,1.055615,1.422293
3,588015508213989555,588015508213989555_8077-6104,2018.1.00541.S,2018,uid://A001/X133d/Xe93,uid://A002/Xd77a17/X1f36,42.032778,-0.752300,0.045791,6.22722,19.448789,1.599746,0.512867
4,587741421099155681,587741421099155681_8241-3703,2018.1.00541.S,2018,uid://A001/X133d/Xe7f,uid://A002/Xd74c3f/X273f,126.461162,18.166697,0.028862,4.86483,20.697749,1.735293,0.542230
...,...,...,...,...,...,...,...,...,...,...,...,...,...
293,587729160054833498,AGC252580,2015.1.00405.S,2015,uid://A001/X2fe/X173,uid://A002/Xb08ef9/X104f,228.405346,4.322536,0.175406,3.60299,16.261699,1.208168,0.581384
294,587726102556180606,AGC242091,2015.1.00405.S,2015,uid://A001/X2fe/X17b,uid://A002/Xb07033/X845,211.344687,5.470736,0.195394,3.12167,16.272983,1.593424,0.524738
295,587727943492698312,AGC191728,2015.1.00405.S,2015,uid://A001/X2fe/X17f,uid://A002/Xb020f7/X1520,139.987509,1.647670,0.176348,2.02607,14.899537,1.549572,0.552635
296,587741533840670747,587741533840670747_8241-6102,2021.1.00094.S,2021,uid://A001/X1590/X344f,uid://A002/Xf656ba/X7725,126.059633,17.331963,0.037222,5.11312,27.678542,3.015737,1.102648


## 2. Load in Molecular Gas Masses computed from ALMA Cubes

In [4]:
fpath = '/arc/projects/salvage/ALMA_reduction/'
file = 'salvage-AGN-Oct10-sample_match-lt-4rp_mrs-gt-2rp_qa2-pass_no-inf-dups_ellipap.out'
file = 'salvage-AGN-Oct14-sample_match-lt-4rp_mrs-gt-2rp_qa2-pass_no-inf-dups_ellipap_iter.out'
file = 'salvage-AGN-Oct16-sample_match-lt-4rp_mrs-gt-2rp_qa2-pass_no-inf-dups_ellipap_iter.out'

objID_masses = np.loadtxt(fpath+file, dtype = str, usecols = [0], unpack = True)
MH2_inner, MH2_outer, MH2_total, MH2_inner_err, MH2_outer_err, MH2_total_err, f_inner, f_outer = np.loadtxt(fpath+file, dtype = float, usecols = [1,2,3,4,5,6,7,8], unpack = True)

In [5]:
d2 = {'objID': objID_masses, 'MH2_total': MH2_total, 'MH2_total_err': MH2_total_err, 'MH2_inner': MH2_inner, 'MH2_inner_err': MH2_inner_err, 'MH2_outer': MH2_outer, 'MH2_outer_err': MH2_outer_err, 'f_inner': f_inner, 'f_outer': f_outer}
df2 = pd.DataFrame(data=d2)

# join the catalogues and overwrite the dataframe
df = df.set_index('objID').join(df2.set_index('objID'), on = 'objID')
df

,source_name,project_code,year,MUID,AUID,ra,dec,z,rpetro,MRS,resolution,sensitivity,MH2_total,MH2_total_err,MH2_inner,MH2_inner_err,MH2_outer,MH2_outer_err,f_inner,f_outer
objID,,,,,,,,,,,,,,,,,,,,
587726033335550094,Abell,2011.0.00374.S,2011,uid://A002/X327408/X22f,uid://A002/X3b3400/X64c,210.258631,2.878469,0.251876,2.91803,10.534628,1.333456,1.500086,10.694705,9.111561,10.317708,8.258747,10.433301,9.025174,1.0,0.018217
587741490891325518,587741490891325518_8241-3704,2018.1.00541.S,2018,uid://A001/X133d/Xe83,uid://A002/Xd74c3f/X24ad,126.568894,17.362436,0.065923,4.65495,21.589244,1.802884,0.914182,10.161126,8.752097,9.419450,7.696260,10.074755,8.743220,1.0,0.050033
587722984441250023,HATLAS_J143336.7+005042,2017.1.00025.S,2017,uid://A001/X1284/X1534,uid://A002/Xca795f/X25d1,218.403317,0.845238,0.069836,1.43384,14.286183,1.055615,1.422293,9.047575,8.353168,9.143029,8.238871,8.966115,8.406747,0.0,0.000000
588015508213989555,588015508213989555_8077-6104,2018.1.00541.S,2018,uid://A001/X133d/Xe93,uid://A002/Xd77a17/X1f36,42.032778,-0.752300,0.045791,6.22722,19.448789,1.599746,0.512867,9.708711,8.140326,8.801124,7.023797,9.652317,8.162352,1.0,0.082839
587741421099155681,587741421099155681_8241-3703,2018.1.00541.S,2018,uid://A001/X133d/Xe7f,uid://A002/Xd74c3f/X273f,126.461162,18.166697,0.028862,4.86483,20.697749,1.735293,0.542230,9.032488,7.592118,8.077017,6.588560,8.981934,7.601914,1.0,0.075229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
587729160054833498,AGC252580,2015.1.00405.S,2015,uid://A001/X2fe/X173,uid://A002/Xb08ef9/X104f,228.405346,4.322536,0.175406,3.60299,16.261699,1.208168,0.581384,-inf,-inf,8.962294,8.286730,-inf,-inf,0.0,0.000000
587726102556180606,AGC242091,2015.1.00405.S,2015,uid://A001/X2fe/X17b,uid://A002/Xb07033/X845,211.344687,5.470736,0.195394,3.12167,16.272983,1.593424,0.524738,10.117028,9.084810,9.528769,8.133164,9.969697,9.082740,1.0,0.088395
587727943492698312,AGC191728,2015.1.00405.S,2015,uid://A001/X2fe/X17f,uid://A002/Xb020f7/X1520,139.987509,1.647670,0.176348,2.02607,14.899537,1.549572,0.552635,-inf,-inf,-inf,-inf,-inf,-inf,0.0,0.000000


## 3. Add visual classification of SALVAGE images and spectra

In [6]:
fpath = '/arc/projects/salvage/ALMA_reduction/'
file = 'SALVAGE_classification.csv'

objID_classes, morph_class, spec_class = np.loadtxt(fpath+file, dtype = str, unpack = True, delimiter = ',', usecols = [0,1,2], skiprows = 1)

In [7]:
d_tmp = {'objID': objID_classes, 'visual_class': morph_class, 'spec_class': spec_class}
df_tmp = pd.DataFrame(data=d_tmp)

# join the catalogues
df = df.join(df_tmp.set_index('objID'), on = 'objID')
df

,source_name,project_code,year,MUID,AUID,ra,dec,z,rpetro,MRS,...,MH2_total,MH2_total_err,MH2_inner,MH2_inner_err,MH2_outer,MH2_outer_err,f_inner,f_outer,visual_class,spec_class
objID,,,,,,,,,,,,,,,,,,,,,
587726033335550094,Abell,2011.0.00374.S,2011,uid://A002/X327408/X22f,uid://A002/X3b3400/X64c,210.258631,2.878469,0.251876,2.91803,10.534628,...,10.694705,9.111561,10.317708,8.258747,10.433301,9.025174,1.0,0.018217,N,A
587741490891325518,587741490891325518_8241-3704,2018.1.00541.S,2018,uid://A001/X133d/Xe83,uid://A002/Xd74c3f/X24ad,126.568894,17.362436,0.065923,4.65495,21.589244,...,10.161126,8.752097,9.419450,7.696260,10.074755,8.743220,1.0,0.050033,N,A
587722984441250023,HATLAS_J143336.7+005042,2017.1.00025.S,2017,uid://A001/X1284/X1534,uid://A002/Xca795f/X25d1,218.403317,0.845238,0.069836,1.43384,14.286183,...,9.047575,8.353168,9.143029,8.238871,8.966115,8.406747,0.0,0.000000,N,C
588015508213989555,588015508213989555_8077-6104,2018.1.00541.S,2018,uid://A001/X133d/Xe93,uid://A002/Xd77a17/X1f36,42.032778,-0.752300,0.045791,6.22722,19.448789,...,9.708711,8.140326,8.801124,7.023797,9.652317,8.162352,1.0,0.082839,N,A
587741421099155681,587741421099155681_8241-3703,2018.1.00541.S,2018,uid://A001/X133d/Xe7f,uid://A002/Xd74c3f/X273f,126.461162,18.166697,0.028862,4.86483,20.697749,...,9.032488,7.592118,8.077017,6.588560,8.981934,7.601914,1.0,0.075229,N,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
587729160054833498,AGC252580,2015.1.00405.S,2015,uid://A001/X2fe/X173,uid://A002/Xb08ef9/X104f,228.405346,4.322536,0.175406,3.60299,16.261699,...,-inf,-inf,8.962294,8.286730,-inf,-inf,0.0,0.000000,N,D
587726102556180606,AGC242091,2015.1.00405.S,2015,uid://A001/X2fe/X17b,uid://A002/Xb07033/X845,211.344687,5.470736,0.195394,3.12167,16.272983,...,10.117028,9.084810,9.528769,8.133164,9.969697,9.082740,1.0,0.088395,P,B
587727943492698312,AGC191728,2015.1.00405.S,2015,uid://A001/X2fe/X17f,uid://A002/Xb020f7/X1520,139.987509,1.647670,0.176348,2.02607,14.899537,...,-inf,-inf,-inf,-inf,-inf,-inf,0.0,0.000000,N,C


## 4. Basic SDSS parameters from local UVic DR7 catalogues

In [8]:
#Query MySQL
db = pymysql.connect(host = 'lauca.phys.uvic.ca', db = 'sdss', user = 'swilkinson', passwd = '123Sdss!@#')

x = 'SELECT u.objID, u.total_mass_med, u.fiber_mass_med, u.total_sfr_med, u.fiber_sfr_med, u.z_spec,\
            u.agn_sn, u.new_K03_flag, u.new_K01_flag, u.agn_sn_mw, u.S06_flag_mw, u.K03_flag_mw, u.K01_flag_mw, u.vdisp, u.balmer_sn\
     FROM dr7_uberuber u WHERE u.total_mass_med > 1'

c = db.cursor()
c.execute(x)
db_data = c.fetchall()
c.close()
db.close()
	
# transpose queried table
db_data = np.array(db_data, dtype = str).T

table_id = db_data[0]

# inner and outer masses
mass_tot = np.array(db_data[1], dtype = float)
mass_fib = np.array(db_data[2], dtype = float)
sfr_tot = np.array(db_data[3], dtype = float)
sfr_fib = np.array(db_data[4], dtype = float)
mass_out = np.log10(10**mass_tot - 10**mass_fib)
sfr_out = np.log10(10**sfr_tot - 10**sfr_fib)

# spectroscopic redshift
z_sdss = np.array(db_data[5], dtype = float)

# Optical AGN classifications
agn_sn = np.array(db_data[6], dtype = str)
agn_sn[agn_sn=='None'] = -1
agn_sn = np.array(agn_sn, dtype = float)
K03 = np.array(db_data[7], dtype = str)
K01 = np.array(db_data[8], dtype = str)

# Optical AGN classifications (corrected using mw extinction curve)
agn_sn_mw = np.array(db_data[9], dtype = str)
agn_sn_mw[agn_sn_mw=='None'] = -1
agn_sn_mw = np.array(agn_sn_mw, dtype = float)
S06_mw = np.array(db_data[10], dtype = str)
K03_mw = np.array(db_data[11], dtype = str)
K01_mw = np.array(db_data[12], dtype = str)

# velocity dispersion
vdisp = np.array(db_data[13], dtype = float)

# balmer_sn
balmer_sn = np.array(db_data[14], dtype = str)
balmer_sn[balmer_sn=='None'] = -1
balmer_sn = np.array(balmer_sn, dtype = float)

/tmp/ipykernel_28198/4047331550.py:24: RuntimeWarning: divide by zero encountered in log10
  mass_out = np.log10(10**mass_tot - 10**mass_fib)
/tmp/ipykernel_28198/4047331550.py:24: RuntimeWarning: invalid value encountered in log10
  mass_out = np.log10(10**mass_tot - 10**mass_fib)
/tmp/ipykernel_28198/4047331550.py:25: RuntimeWarning: invalid value encountered in log10
  sfr_out = np.log10(10**sfr_tot - 10**sfr_fib)


In [9]:
# create a mw-corrected hybrid, where lines are only mw-corrected if balmer_sn > 3

agn_sn_mw_bsn = agn_sn_mw.copy()
agn_sn_mw_bsn[balmer_sn<3] = agn_sn[balmer_sn<3]

K03_mw_bsn = K03_mw.copy()
K03_mw_bsn[balmer_sn<3] = K03[balmer_sn<3]

K01_mw_bsn = K01_mw.copy()
K01_mw_bsn[balmer_sn<3] = K01[balmer_sn<3]

In [10]:
d_tmp = {'objID': table_id, 'mstar_inner': mass_fib, 'sfr_inner': sfr_fib, 'mstar_outer': mass_out, 'sfr_outer': sfr_out, 'mstar_total': mass_tot, 'sfr_total': sfr_tot, \
      'K03_lauca': K03, 'K01_lauca': K01, 'agn_sn': agn_sn, 'balmer_sn': balmer_sn, \
      'S06_lauca_mw': S06_mw, 'K03_lauca_mw': K03_mw, 'K01_lauca_mw': K01_mw, 'agn_sn_mw': agn_sn_mw, \
      'K03_lauca_mw_bsn': K03_mw_bsn, 'K01_lauca_mw_bsn': K01_mw_bsn, 'agn_sn_mw_bsn': agn_sn_mw_bsn, \
      'vdisp': vdisp}
df_tmp = pd.DataFrame(data=d_tmp)

# join the catalogues and overwrite dataframe
df = df.join(df_tmp.set_index('objID'), on = 'objID')
df

,source_name,project_code,year,MUID,AUID,ra,dec,z,rpetro,MRS,...,agn_sn,balmer_sn,S06_lauca_mw,K03_lauca_mw,K01_lauca_mw,agn_sn_mw,K03_lauca_mw_bsn,K01_lauca_mw_bsn,agn_sn_mw_bsn,vdisp
objID,,,,,,,,,,,,,,,,,,,,,
587726033335550094,Abell,2011.0.00374.S,2011,uid://A002/X327408/X22f,uid://A002/X3b3400/X64c,210.258631,2.878469,0.251876,2.91803,10.534628,...,-1.00000,-1.00000,None,None,None,-1.00000,NULL,NULL,-1.00000,-9999.000
587741490891325518,587741490891325518_8241-3704,2018.1.00541.S,2018,uid://A001/X133d/Xe83,uid://A002/Xd74c3f/X24ad,126.568894,17.362436,0.065923,4.65495,21.589244,...,4.63888,16.28180,SF,SF,SF,7.26448,SF,SF,7.26448,-9999.000
587722984441250023,HATLAS_J143336.7+005042,2017.1.00025.S,2017,uid://A001/X1284/X1534,uid://A002/Xca795f/X25d1,218.403317,0.845238,0.069836,1.43384,14.286183,...,6.15106,17.71090,SF,SF,SF,9.63253,SF,SF,9.63253,-9999.000
588015508213989555,588015508213989555_8077-6104,2018.1.00541.S,2018,uid://A001/X133d/Xe93,uid://A002/Xd77a17/X1f36,42.032778,-0.752300,0.045791,6.22722,19.448789,...,4.57055,9.70241,AGN,SF,SF,7.15745,SF,SF,7.15745,-9999.000
587741421099155681,587741421099155681_8241-3703,2018.1.00541.S,2018,uid://A001/X133d/Xe7f,uid://A002/Xd74c3f/X273f,126.461162,18.166697,0.028862,4.86483,20.697749,...,8.09660,21.57830,SF,SF,SF,12.67930,SF,SF,12.67930,-9999.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
587729160054833498,AGC252580,2015.1.00405.S,2015,uid://A001/X2fe/X173,uid://A002/Xb08ef9/X104f,228.405346,4.322536,0.175406,3.60299,16.261699,...,1.06031,1.56586,AGN,SF,SF,1.66045,SF,SF,1.06031,-9999.000
587726102556180606,AGC242091,2015.1.00405.S,2015,uid://A001/X2fe/X17b,uid://A002/Xb07033/X845,211.344687,5.470736,0.195394,3.12167,16.272983,...,2.66823,2.70292,SF,SF,SF,4.17847,SF,SF,2.66823,167.263
587727943492698312,AGC191728,2015.1.00405.S,2015,uid://A001/X2fe/X17f,uid://A002/Xb020f7/X1520,139.987509,1.647670,0.176348,2.02607,14.899537,...,9.58997,22.48170,AGN,SF,SF,15.01790,SF,SF,15.01790,-9999.000


## 5. Adding Line Flux Measurements

In [11]:
#Query MySQL
db = pymysql.connect(host = 'lauca.phys.uvic.ca', db = 'sdss', user = 'swilkinson', passwd = '123Sdss!@#')

x = 'SELECT e.objID, e.Halpha_flux, e.Halpha_cont, e.Hbeta_flux, e.OIII_5007_flux, NII_flux, OII_flux, OI_flux, corr_Halpha_flux_mw, corr_Hbeta_flux_mw, corr_OIII_5007_flux_mw, corr_NII_flux_mw, corr_OII_flux_mw, corr_OI_flux_mw  \
     FROM dr7_emmlines e'

c = db.cursor()
c.execute(x)
db_data = c.fetchall()
c.close()
db.close()
	
# transpose queried table
db_data = np.array(db_data, dtype = str).T

table_id = db_data[0]

# Ha flux
Ha_flux = np.array(db_data[1], dtype = float)
Ha_cont  = np.array(db_data[2], dtype = float)
Ha_eqw = Ha_flux/Ha_cont

# other emission lines (not MW corrected!)
Hb_flux   = np.array(db_data[3], dtype = float)
OIII_flux = np.array(db_data[4], dtype = float)
NII_flux  = np.array(db_data[5], dtype = float)
OII_flux  = np.array(db_data[6], dtype = float)
OI_flux   = np.array(db_data[7], dtype = float)

# other emission lines (mw corrected... ?)
Ha_flux_mw  = np.array(db_data[8], dtype = str)
Hb_flux_mw  = np.array(db_data[9], dtype = str)
OIII_flux_mw = np.array(db_data[10], dtype = str)
NII_flux_mw  = np.array(db_data[11], dtype = str)
OII_flux_mw  = np.array(db_data[12], dtype = str)
OI_flux_mw  = np.array(db_data[13], dtype = str)
Ha_flux_mw[Ha_flux_mw=='None'] = -1
Hb_flux_mw[Hb_flux_mw=='None'] = -1
OIII_flux_mw[OIII_flux_mw=='None'] = -1
NII_flux_mw[NII_flux_mw=='None'] = -1
OII_flux_mw[OII_flux_mw=='None'] = -1
OI_flux_mw[OI_flux_mw=='None'] = -1
Ha_flux_mw = np.array(Ha_flux_mw, dtype = float)
Hb_flux_mw = np.array(Hb_flux_mw, dtype = float)
OIII_flux_mw = np.array(OIII_flux_mw, dtype = float)
NII_flux_mw = np.array(NII_flux_mw, dtype = float)
OII_flux_mw = np.array(OII_flux_mw, dtype = float)
OI_flux_mw = np.array(OI_flux_mw, dtype = float)

/tmp/ipykernel_28198/2421302734.py:21: RuntimeWarning: divide by zero encountered in divide
  Ha_eqw = Ha_flux/Ha_cont
/tmp/ipykernel_28198/2421302734.py:21: RuntimeWarning: invalid value encountered in divide
  Ha_eqw = Ha_flux/Ha_cont


In [12]:
d_tmp = {'objID': table_id, 'Ha_flux': Ha_flux, 'Hb_flux': Hb_flux, 'OIII_flux': OIII_flux, 'NII_flux': NII_flux, 'OII_flux': OII_flux, 'OI_flux': OI_flux, 'Ha_eqw': -Ha_eqw,\
      'Ha_flux_mw': Ha_flux_mw, 'Hb_flux_mw': Hb_flux_mw, 'OIII_flux_mw': OIII_flux_mw, 'NII_flux_mw': NII_flux_mw, 'OII_flux_mw': OII_flux_mw, 'OI_flux_mw': OI_flux_mw}
df_tmp = pd.DataFrame(data=d_tmp)

# join the catalogues and overwrite dataframe
df = df.join(df_tmp.set_index('objID'), on = 'objID')
df

,source_name,project_code,year,MUID,AUID,ra,dec,z,rpetro,MRS,...,NII_flux,OII_flux,OI_flux,Ha_eqw,Ha_flux_mw,Hb_flux_mw,OIII_flux_mw,NII_flux_mw,OII_flux_mw,OI_flux_mw
objID,,,,,,,,,,,,,,,,,,,,,
587726033335550094,Abell,2011.0.00374.S,2011,uid://A002/X327408/X22f,uid://A002/X3b3400/X64c,210.258631,2.878469,0.251876,2.91803,10.534628,...,781.3730,692.4690,192.62400,-52.609165,-1.000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000
587741490891325518,587741490891325518_8241-3704,2018.1.00541.S,2018,uid://A001/X133d/Xe83,uid://A002/Xd74c3f/X24ad,126.568894,17.362436,0.065923,4.65495,21.589244,...,368.8010,144.8560,30.33670,-22.125499,3146.440,1105.8900,207.1040,1237.8400,1468.8500,109.7380
587722984441250023,HATLAS_J143336.7+005042,2017.1.00025.S,2017,uid://A001/X1284/X1534,uid://A002/Xca795f/X25d1,218.403317,0.845238,0.069836,1.43384,14.286183,...,179.2140,127.3390,10.97240,-30.779849,1273.130,447.3210,86.7264,472.3860,813.4510,30.7082
588015508213989555,588015508213989555_8077-6104,2018.1.00541.S,2018,uid://A001/X133d/Xe93,uid://A002/Xd77a17/X1f36,42.032778,-0.752300,0.045791,6.22722,19.448789,...,111.6300,46.5046,13.10890,-11.006645,727.790,255.7130,81.5136,294.2220,297.0730,36.6846
587741421099155681,587741421099155681_8241-3703,2018.1.00541.S,2018,uid://A001/X133d/Xe7f,uid://A002/Xd74c3f/X273f,126.461162,18.166697,0.028862,4.86483,20.697749,...,201.4010,193.5510,14.02040,-25.880058,1006.260,353.3250,73.1148,334.2270,510.0790,24.0073
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
587729160054833498,AGC252580,2015.1.00405.S,2015,uid://A001/X2fe/X173,uid://A002/Xb08ef9/X104f,228.405346,4.322536,0.175406,3.60299,16.261699,...,18.4922,10.6565,6.96273,-9.263839,75.963,26.6776,10.8925,34.9773,22.3507,13.6991
587726102556180606,AGC242091,2015.1.00405.S,2015,uid://A001/X2fe/X17b,uid://A002/Xb07033/X845,211.344687,5.470736,0.195394,3.12167,16.272983,...,30.3310,27.5034,2.80994,-13.733403,384.988,135.3270,61.8395,109.0840,318.4780,10.9382
587727943492698312,AGC191728,2015.1.00405.S,2015,uid://A001/X2fe/X17f,uid://A002/Xb020f7/X1520,139.987509,1.647670,0.176348,2.02607,14.899537,...,184.2120,175.0160,17.43060,-36.466129,984.064,345.6430,133.1230,384.5100,715.2980,38.0771


## 6. Adding WISE magnitudes and mid-IR AGN classification

In [13]:
#Query MySQL
db = pymysql.connect(host = 'lauca.phys.uvic.ca', db = 'sdss', user = 'swilkinson', passwd = '123Sdss!@#')

x = 'SELECT w.objID, w.W1, w.W2, w.W1_sn, w.W2_sn\
    FROM dr7_unwise w'

c = db.cursor()
c.execute(x)
db_data = c.fetchall()
c.close()
db.close()
	
# transpose queried table
db_data = np.array(db_data, dtype = str).T

table_id = db_data[0]

# Mid-IR AGN Classifications
W1 = np.array(db_data[1], dtype = str)
W2 = np.array(db_data[2], dtype = str)
W1_sn = np.array(db_data[3], dtype = str)
W2_sn = np.array(db_data[4], dtype = str)
W1[W1=='None'] = -1
W2[W2=='None'] = -1
W1_sn[W1_sn=='None'] = -1
W2_sn[W2_sn=='None'] = -1
W1 = np.array(W1, dtype = float)
W2 = np.array(W2, dtype = float)
W1_sn = np.array(W1_sn, dtype = float)
W2_sn = np.array(W2_sn, dtype = float)

WISE_flag = W1-W2>0.5

In [14]:
d_tmp = {'objID': table_id, 'W1': W1, 'W2': W2, 'W1_sn': W1_sn, 'W2_sn': W2_sn, 'WISE_flag': WISE_flag}
df_tmp = pd.DataFrame(data=d_tmp)

# join the catalogues and overwrite dataframe
df = df.join(df_tmp.set_index('objID'), on = 'objID')
df

,source_name,project_code,year,MUID,AUID,ra,dec,z,rpetro,MRS,...,Hb_flux_mw,OIII_flux_mw,NII_flux_mw,OII_flux_mw,OI_flux_mw,W1,W2,W1_sn,W2_sn,WISE_flag
objID,,,,,,,,,,,,,,,,,,,,,
587726033335550094,Abell,2011.0.00374.S,2011,uid://A002/X327408/X22f,uid://A002/X3b3400/X64c,210.258631,2.878469,0.251876,2.91803,10.534628,...,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,12.187465,11.600580,535.61470,260.731700,True
587741490891325518,587741490891325518_8241-3704,2018.1.00541.S,2018,uid://A001/X133d/Xe83,uid://A002/Xd74c3f/X24ad,126.568894,17.362436,0.065923,4.65495,21.589244,...,1105.8900,207.1040,1237.8400,1468.8500,109.7380,11.546753,11.475962,631.47076,196.292510,False
587722984441250023,HATLAS_J143336.7+005042,2017.1.00025.S,2017,uid://A001/X1284/X1534,uid://A002/Xca795f/X25d1,218.403317,0.845238,0.069836,1.43384,14.286183,...,447.3210,86.7264,472.3860,813.4510,30.7082,13.817814,13.563032,220.30084,70.515335,False
588015508213989555,588015508213989555_8077-6104,2018.1.00541.S,2018,uid://A001/X133d/Xe93,uid://A002/Xd77a17/X1f36,42.032778,-0.752300,0.045791,6.22722,19.448789,...,255.7130,81.5136,294.2220,297.0730,36.6846,11.868240,11.676346,569.11090,174.706770,False
587741421099155681,587741421099155681_8241-3703,2018.1.00541.S,2018,uid://A001/X133d/Xe7f,uid://A002/Xd74c3f/X273f,126.461162,18.166697,0.028862,4.86483,20.697749,...,353.3250,73.1148,334.2270,510.0790,24.0073,12.425040,12.259258,309.76556,105.421646,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
587729160054833498,AGC252580,2015.1.00405.S,2015,uid://A001/X2fe/X173,uid://A002/Xb08ef9/X104f,228.405346,4.322536,0.175406,3.60299,16.261699,...,26.6776,10.8925,34.9773,22.3507,13.6991,14.409816,14.126646,73.03163,27.655120,False
587726102556180606,AGC242091,2015.1.00405.S,2015,uid://A001/X2fe/X17b,uid://A002/Xb07033/X845,211.344687,5.470736,0.195394,3.12167,16.272983,...,135.3270,61.8395,109.0840,318.4780,10.9382,14.059337,13.778737,126.05365,46.385544,False
587727943492698312,AGC191728,2015.1.00405.S,2015,uid://A001/X2fe/X17f,uid://A002/Xb020f7/X1520,139.987509,1.647670,0.176348,2.02607,14.899537,...,345.6430,133.1230,384.5100,715.2980,38.0771,13.903738,13.464678,147.59657,61.714794,False


## 7. Adding LERG/HERG flag

In [15]:
#Query MySQL
db = pymysql.connect(host = 'lauca.phys.uvic.ca', db = 'sdss', user = 'swilkinson', passwd = '123Sdss!@#')
x = 'SELECT b.objID, b.HERG, b.LERG\
     FROM dr7_radio_Best b'
c = db.cursor()
c.execute(x)
db_data = c.fetchall()
c.close()
db.close()

# transpose queried table
db_data = np.array(db_data, dtype = str).T

table_id_best = db_data[0]
HERG = np.array(db_data[1], dtype = float)
LERG = np.array(db_data[2], dtype = float)

In [16]:
d_tmp = {'objID': table_id_best, 'LERG_flag': LERG, 'HERG_flag': HERG}
df_tmp = pd.DataFrame(data=d_tmp)

# join the catalogues and overwrite dataframe
df = df.join(df_tmp.set_index('objID'), on = 'objID')
df

,source_name,project_code,year,MUID,AUID,ra,dec,z,rpetro,MRS,...,NII_flux_mw,OII_flux_mw,OI_flux_mw,W1,W2,W1_sn,W2_sn,WISE_flag,LERG_flag,HERG_flag
objID,,,,,,,,,,,,,,,,,,,,,
587726033335550094,Abell,2011.0.00374.S,2011,uid://A002/X327408/X22f,uid://A002/X3b3400/X64c,210.258631,2.878469,0.251876,2.91803,10.534628,...,-1.0000,-1.0000,-1.0000,12.187465,11.600580,535.61470,260.731700,True,1.0,0.0
587741490891325518,587741490891325518_8241-3704,2018.1.00541.S,2018,uid://A001/X133d/Xe83,uid://A002/Xd74c3f/X24ad,126.568894,17.362436,0.065923,4.65495,21.589244,...,1237.8400,1468.8500,109.7380,11.546753,11.475962,631.47076,196.292510,False,NaN,NaN
587722984441250023,HATLAS_J143336.7+005042,2017.1.00025.S,2017,uid://A001/X1284/X1534,uid://A002/Xca795f/X25d1,218.403317,0.845238,0.069836,1.43384,14.286183,...,472.3860,813.4510,30.7082,13.817814,13.563032,220.30084,70.515335,False,NaN,NaN
588015508213989555,588015508213989555_8077-6104,2018.1.00541.S,2018,uid://A001/X133d/Xe93,uid://A002/Xd77a17/X1f36,42.032778,-0.752300,0.045791,6.22722,19.448789,...,294.2220,297.0730,36.6846,11.868240,11.676346,569.11090,174.706770,False,NaN,NaN
587741421099155681,587741421099155681_8241-3703,2018.1.00541.S,2018,uid://A001/X133d/Xe7f,uid://A002/Xd74c3f/X273f,126.461162,18.166697,0.028862,4.86483,20.697749,...,334.2270,510.0790,24.0073,12.425040,12.259258,309.76556,105.421646,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
587729160054833498,AGC252580,2015.1.00405.S,2015,uid://A001/X2fe/X173,uid://A002/Xb08ef9/X104f,228.405346,4.322536,0.175406,3.60299,16.261699,...,34.9773,22.3507,13.6991,14.409816,14.126646,73.03163,27.655120,False,NaN,NaN
587726102556180606,AGC242091,2015.1.00405.S,2015,uid://A001/X2fe/X17b,uid://A002/Xb07033/X845,211.344687,5.470736,0.195394,3.12167,16.272983,...,109.0840,318.4780,10.9382,14.059337,13.778737,126.05365,46.385544,False,NaN,NaN
587727943492698312,AGC191728,2015.1.00405.S,2015,uid://A001/X2fe/X17f,uid://A002/Xb020f7/X1520,139.987509,1.647670,0.176348,2.02607,14.899537,...,384.5100,715.2980,38.0771,13.903738,13.464678,147.59657,61.714794,False,NaN,NaN


## 8. Adding Sersic fit disk inclination and B/T

In [17]:
#Query MySQL
db = pymysql.connect(host = 'lauca.phys.uvic.ca', db = 'sdss', user = 'swilkinson', passwd = '123Sdss!@#')

x = 'SELECT m.objID, m.incd, m.bt_r, m.bt_g\
     FROM sdss_dr7_morph_mybkg_mydeblend_gr_pub2011 m'

c = db.cursor()
c.execute(x)
db_data = c.fetchall()
c.close()
db.close()
	
# transpose queried table
db_data = np.array(db_data, dtype = str).T

table_id = db_data[0]

# orientation
incl = np.array(db_data[1], dtype = str)
incl[incl=='None'] = -1
incl = np.array(incl, dtype = float)

# g-band B/T
btg = np.array(db_data[3], dtype = str)
btg[btg=='None'] = -1
btg = np.array(btg, dtype = float)

# r-band B/T
bt = np.array(db_data[2], dtype = str)
bt[bt=='None'] = -1
bt = np.array(bt, dtype = float)

In [18]:
d_tmp = {'objID': table_id, 'incl': incl, 'bt': bt}
df_tmp = pd.DataFrame(data=d_tmp)

# join the catalogues and overwrite dataframe
df = df.join(df_tmp.set_index('objID'), on = 'objID')
df

,source_name,project_code,year,MUID,AUID,ra,dec,z,rpetro,MRS,...,OI_flux_mw,W1,W2,W1_sn,W2_sn,WISE_flag,LERG_flag,HERG_flag,incl,bt
objID,,,,,,,,,,,,,,,,,,,,,
587726033335550094,Abell,2011.0.00374.S,2011,uid://A002/X327408/X22f,uid://A002/X3b3400/X64c,210.258631,2.878469,0.251876,2.91803,10.534628,...,-1.0000,12.187465,11.600580,535.61470,260.731700,True,1.0,0.0,24.980,0.412
587741490891325518,587741490891325518_8241-3704,2018.1.00541.S,2018,uid://A001/X133d/Xe83,uid://A002/Xd74c3f/X24ad,126.568894,17.362436,0.065923,4.65495,21.589244,...,109.7380,11.546753,11.475962,631.47076,196.292510,False,NaN,NaN,29.165,0.096
587722984441250023,HATLAS_J143336.7+005042,2017.1.00025.S,2017,uid://A001/X1284/X1534,uid://A002/Xca795f/X25d1,218.403317,0.845238,0.069836,1.43384,14.286183,...,30.7082,13.817814,13.563032,220.30084,70.515335,False,NaN,NaN,67.125,0.267
588015508213989555,588015508213989555_8077-6104,2018.1.00541.S,2018,uid://A001/X133d/Xe93,uid://A002/Xd77a17/X1f36,42.032778,-0.752300,0.045791,6.22722,19.448789,...,36.6846,11.868240,11.676346,569.11090,174.706770,False,NaN,NaN,16.805,0.062
587741421099155681,587741421099155681_8241-3703,2018.1.00541.S,2018,uid://A001/X133d/Xe7f,uid://A002/Xd74c3f/X273f,126.461162,18.166697,0.028862,4.86483,20.697749,...,24.0073,12.425040,12.259258,309.76556,105.421646,False,NaN,NaN,21.294,0.158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
587729160054833498,AGC252580,2015.1.00405.S,2015,uid://A001/X2fe/X173,uid://A002/Xb08ef9/X104f,228.405346,4.322536,0.175406,3.60299,16.261699,...,13.6991,14.409816,14.126646,73.03163,27.655120,False,NaN,NaN,62.798,0.089
587726102556180606,AGC242091,2015.1.00405.S,2015,uid://A001/X2fe/X17b,uid://A002/Xb07033/X845,211.344687,5.470736,0.195394,3.12167,16.272983,...,10.9382,14.059337,13.778737,126.05365,46.385544,False,NaN,NaN,64.668,0.099
587727943492698312,AGC191728,2015.1.00405.S,2015,uid://A001/X2fe/X17f,uid://A002/Xb020f7/X1520,139.987509,1.647670,0.176348,2.02607,14.899537,...,38.0771,13.903738,13.464678,147.59657,61.714794,False,NaN,NaN,0.640,0.356


## 9. Add SDSS photometric model parameters

In [19]:
objID_sdss = np.loadtxt('/arc/projects/salvage/ALMA_reduction/model_params_swilkinson_0.csv', delimiter = ',', unpack = True, usecols = [0], dtype = str, skiprows = 1)
r_DeV, ab_DeV, phi_DeV, L_DeV, r_exp, ab_exp, phi_exp, L_exp = np.loadtxt('/arc/projects/salvage/ALMA_reduction/model_params_swilkinson_0.csv', delimiter = ',', unpack = True, usecols = [1,2,3,4,5,6,7,8], dtype = float, skiprows = 1)

In [20]:
R_A, R_B, PHI, TYPE = [], [], [], []

for i in range(len(objID_sdss)):


    if L_exp[i] > L_DeV[i]:

        R_A.append(r_exp[i] * 4)
        R_B.append(r_exp[i] * ab_exp[i] * 4)
        PHI.append(phi_exp[i] - 90)
        TYPE.append('EXP')

    else: 

        R_A.append(r_DeV[i] * 8)
        R_B.append(r_DeV[i] * ab_DeV[i] * 8)
        PHI.append(phi_DeV[i] - 90)
        TYPE.append('DeV')

In [21]:
d_tmp = {'objID': objID_sdss, 'r_a_outer': R_A, 'r_b_outer': R_B, 'phi_outer': PHI, 'outer_ap_type': TYPE}
df_tmp = pd.DataFrame(data=d_tmp)

# join the catalogues and overwrite dataframe
df = df.join(df_tmp.set_index('objID'), on = 'objID').drop_duplicates()
df

,source_name,project_code,year,MUID,AUID,ra,dec,z,rpetro,MRS,...,W2_sn,WISE_flag,LERG_flag,HERG_flag,incl,bt,r_a_outer,r_b_outer,phi_outer,outer_ap_type
objID,,,,,,,,,,,,,,,,,,,,,
587726033335550094,Abell,2011.0.00374.S,2011,uid://A002/X327408/X22f,uid://A002/X3b3400/X64c,210.258631,2.878469,0.251876,2.91803,10.534628,...,260.731700,True,1.0,0.0,24.980,0.412,31.042336,24.455481,71.585200,DeV
587741490891325518,587741490891325518_8241-3704,2018.1.00541.S,2018,uid://A001/X133d/Xe83,uid://A002/Xd74c3f/X24ad,126.568894,17.362436,0.065923,4.65495,21.589244,...,196.292510,False,NaN,NaN,29.165,0.096,58.420792,52.677818,20.922400,DeV
587722984441250023,HATLAS_J143336.7+005042,2017.1.00025.S,2017,uid://A001/X1284/X1534,uid://A002/Xca795f/X25d1,218.403317,0.845238,0.069836,1.43384,14.286183,...,70.515335,False,NaN,NaN,67.125,0.267,6.158172,2.695376,-10.808490,EXP
588015508213989555,588015508213989555_8077-6104,2018.1.00541.S,2018,uid://A001/X133d/Xe93,uid://A002/Xd77a17/X1f36,42.032778,-0.752300,0.045791,6.22722,19.448789,...,174.706770,False,NaN,NaN,16.805,0.062,24.675532,23.413573,-80.213695,EXP
587741421099155681,587741421099155681_8241-3703,2018.1.00541.S,2018,uid://A001/X133d/Xe7f,uid://A002/Xd74c3f/X273f,126.461162,18.166697,0.028862,4.86483,20.697749,...,105.421646,False,NaN,NaN,21.294,0.158,18.242140,17.204775,53.828600,EXP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
587729160054833498,AGC252580,2015.1.00405.S,2015,uid://A001/X2fe/X173,uid://A002/Xb08ef9/X104f,228.405346,4.322536,0.175406,3.60299,16.261699,...,27.655120,False,NaN,NaN,62.798,0.089,19.473216,8.589100,42.432000,EXP
587726102556180606,AGC242091,2015.1.00405.S,2015,uid://A001/X2fe/X17b,uid://A002/Xb07033/X845,211.344687,5.470736,0.195394,3.12167,16.272983,...,46.385544,False,NaN,NaN,64.668,0.099,16.879012,7.736601,-6.128780,EXP
587727943492698312,AGC191728,2015.1.00405.S,2015,uid://A001/X2fe/X17f,uid://A002/Xb020f7/X1520,139.987509,1.647670,0.176348,2.02607,14.899537,...,61.714794,False,NaN,NaN,0.640,0.356,7.625876,5.178157,3.729450,EXP


## 10. Add physical aperture masses

In [22]:
fpath = '/arc/projects/salvage/ALMA_reduction/output/'
file = 'salvage-AGN-Oct10-sample_match-lt-4rp_mrs-gt-2rp_qa2-pass_no-inf-dups-physap-1kpc-10kpc.out'

objID_masses = np.loadtxt(fpath+file, dtype = str, usecols = [0], unpack = True)
MH2_inner, MH2_outer, MH2_total, MH2_inner_err, MH2_outer_err, MH2_total_err = np.loadtxt(fpath+file, dtype = float, usecols = [1,2,3,4,5,6], unpack = True)

In [23]:
d_tmp = {'objID': objID_masses, 'MH2_10kpc_total': MH2_total, 'MH2_10kpc_total_err': MH2_total_err, 'MH2_1kpc_inner': MH2_inner, 'MH2_1kpc_inner_err': MH2_inner_err, 'MH2_10kpc_outer': MH2_outer, 'MH2_10kpc_outer_err': MH2_outer_err}
df_tmp = pd.DataFrame(data=d_tmp)

# join the catalogues and overwrite dataframe
df = df.join(df_tmp.set_index('objID'), on = 'objID')
df

,source_name,project_code,year,MUID,AUID,ra,dec,z,rpetro,MRS,...,r_a_outer,r_b_outer,phi_outer,outer_ap_type,MH2_10kpc_total,MH2_10kpc_total_err,MH2_1kpc_inner,MH2_1kpc_inner_err,MH2_10kpc_outer,MH2_10kpc_outer_err
objID,,,,,,,,,,,,,,,,,,,,,
587726033335550094,Abell,2011.0.00374.S,2011,uid://A002/X327408/X22f,uid://A002/X3b3400/X64c,210.258631,2.878469,0.251876,2.91803,10.534628,...,31.042336,24.455481,71.585200,DeV,NaN,NaN,NaN,NaN,NaN,NaN
587741490891325518,587741490891325518_8241-3704,2018.1.00541.S,2018,uid://A001/X133d/Xe83,uid://A002/Xd74c3f/X24ad,126.568894,17.362436,0.065923,4.65495,21.589244,...,58.420792,52.677818,20.922400,DeV,NaN,NaN,NaN,NaN,NaN,NaN
587722984441250023,HATLAS_J143336.7+005042,2017.1.00025.S,2017,uid://A001/X1284/X1534,uid://A002/Xca795f/X25d1,218.403317,0.845238,0.069836,1.43384,14.286183,...,6.158172,2.695376,-10.808490,EXP,NaN,NaN,NaN,NaN,NaN,NaN
588015508213989555,588015508213989555_8077-6104,2018.1.00541.S,2018,uid://A001/X133d/Xe93,uid://A002/Xd77a17/X1f36,42.032778,-0.752300,0.045791,6.22722,19.448789,...,24.675532,23.413573,-80.213695,EXP,NaN,NaN,NaN,NaN,NaN,NaN
587741421099155681,587741421099155681_8241-3703,2018.1.00541.S,2018,uid://A001/X133d/Xe7f,uid://A002/Xd74c3f/X273f,126.461162,18.166697,0.028862,4.86483,20.697749,...,18.242140,17.204775,53.828600,EXP,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
587729160054833498,AGC252580,2015.1.00405.S,2015,uid://A001/X2fe/X173,uid://A002/Xb08ef9/X104f,228.405346,4.322536,0.175406,3.60299,16.261699,...,19.473216,8.589100,42.432000,EXP,NaN,NaN,NaN,NaN,NaN,NaN
587726102556180606,AGC242091,2015.1.00405.S,2015,uid://A001/X2fe/X17b,uid://A002/Xb07033/X845,211.344687,5.470736,0.195394,3.12167,16.272983,...,16.879012,7.736601,-6.128780,EXP,NaN,NaN,NaN,NaN,NaN,NaN
587727943492698312,AGC191728,2015.1.00405.S,2015,uid://A001/X2fe/X17f,uid://A002/Xb020f7/X1520,139.987509,1.647670,0.176348,2.02607,14.899537,...,7.625876,5.178157,3.729450,EXP,NaN,NaN,NaN,NaN,NaN,NaN


## 11. Add PSB Classifications

In [24]:
#Query MySQL
db = pymysql.connect(host = 'lauca.phys.uvic.ca', db = 'sdss', user = 'swilkinson', passwd = '123Sdss!@#')

x = 'SELECT p.objID, p.selected_by_PCA, p.selected_by_goto FROM dr7_PSB_params p'

c = db.cursor()
c.execute(x)
db_data = c.fetchall()
c.close()
db.close()
	
# transpose queried table
db_data = np.array(db_data, dtype = str).T

table_id = db_data[0]

# Mid-IR AGN Classifications
pca_psb_flag = np.array(db_data[1], dtype = str)
goto_psb_flag = np.array(db_data[2], dtype = str)

In [25]:
d_tmp = {'objID': table_id, 'pca_psb_flag': pca_psb_flag , 'goto_psb_flag': goto_psb_flag}
df_tmp = pd.DataFrame(data=d_tmp)

# join the catalogues and overwrite dataframe
df = df.join(df_tmp.set_index('objID'), on = 'objID')
df

,source_name,project_code,year,MUID,AUID,ra,dec,z,rpetro,MRS,...,phi_outer,outer_ap_type,MH2_10kpc_total,MH2_10kpc_total_err,MH2_1kpc_inner,MH2_1kpc_inner_err,MH2_10kpc_outer,MH2_10kpc_outer_err,pca_psb_flag,goto_psb_flag
objID,,,,,,,,,,,,,,,,,,,,,
587726033335550094,Abell,2011.0.00374.S,2011,uid://A002/X327408/X22f,uid://A002/X3b3400/X64c,210.258631,2.878469,0.251876,2.91803,10.534628,...,71.585200,DeV,NaN,NaN,NaN,NaN,NaN,NaN,0,None
587741490891325518,587741490891325518_8241-3704,2018.1.00541.S,2018,uid://A001/X133d/Xe83,uid://A002/Xd74c3f/X24ad,126.568894,17.362436,0.065923,4.65495,21.589244,...,20.922400,DeV,NaN,NaN,NaN,NaN,NaN,NaN,2,None
587722984441250023,HATLAS_J143336.7+005042,2017.1.00025.S,2017,uid://A001/X1284/X1534,uid://A002/Xca795f/X25d1,218.403317,0.845238,0.069836,1.43384,14.286183,...,-10.808490,EXP,NaN,NaN,NaN,NaN,NaN,NaN,2,None
588015508213989555,588015508213989555_8077-6104,2018.1.00541.S,2018,uid://A001/X133d/Xe93,uid://A002/Xd77a17/X1f36,42.032778,-0.752300,0.045791,6.22722,19.448789,...,-80.213695,EXP,NaN,NaN,NaN,NaN,NaN,NaN,0,None
587741421099155681,587741421099155681_8241-3703,2018.1.00541.S,2018,uid://A001/X133d/Xe7f,uid://A002/Xd74c3f/X273f,126.461162,18.166697,0.028862,4.86483,20.697749,...,53.828600,EXP,NaN,NaN,NaN,NaN,NaN,NaN,2,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
587729160054833498,AGC252580,2015.1.00405.S,2015,uid://A001/X2fe/X173,uid://A002/Xb08ef9/X104f,228.405346,4.322536,0.175406,3.60299,16.261699,...,42.432000,EXP,NaN,NaN,NaN,NaN,NaN,NaN,0,None
587726102556180606,AGC242091,2015.1.00405.S,2015,uid://A001/X2fe/X17b,uid://A002/Xb07033/X845,211.344687,5.470736,0.195394,3.12167,16.272983,...,-6.128780,EXP,NaN,NaN,NaN,NaN,NaN,NaN,0,None
587727943492698312,AGC191728,2015.1.00405.S,2015,uid://A001/X2fe/X17f,uid://A002/Xb020f7/X1520,139.987509,1.647670,0.176348,2.02607,14.899537,...,3.729450,EXP,NaN,NaN,NaN,NaN,NaN,NaN,2,None


In [26]:
len(df.index[df['goto_psb_flag'] == '1'])

2

In [27]:
len(df.index[df['pca_psb_flag'] == '1'])

5

## adding some final columns.. 

In [28]:
# create a mw-corrected hybrid, where lines are only mw-corrected if balmer_sn > 3

Ha_flux_mw_bsn = df['Ha_flux_mw'].copy()
Ha_flux_mw_bsn[df['balmer_sn']<3] = df['Ha_flux'][df['balmer_sn']<3]
df['Ha_flux_mw_bsn'] = Ha_flux_mw_bsn

Hb_flux_mw_bsn = df['Hb_flux_mw'].copy()
Hb_flux_mw_bsn[df['balmer_sn']<3] = df['Hb_flux'][df['balmer_sn']<3]
df['Hb_flux_mw_bsn'] = Hb_flux_mw_bsn

OIII_flux_mw_bsn = df['OIII_flux_mw'].copy()
OIII_flux_mw_bsn[df['balmer_sn']<3] = df['OIII_flux'][df['balmer_sn']<3]
df['OIII_flux_mw_bsn'] = OIII_flux_mw_bsn

NII_flux_mw_bsn = df['NII_flux_mw'].copy()
NII_flux_mw_bsn[df['balmer_sn']<3] = df['NII_flux'][df['balmer_sn']<3]
df['NII_flux_mw_bsn'] = NII_flux_mw_bsn

OII_flux_mw_bsn = df['OII_flux_mw'].copy()
OII_flux_mw_bsn[df['balmer_sn']<3] = df['OII_flux'][df['balmer_sn']<3]
df['OII_flux_mw_bsn'] = OII_flux_mw_bsn

OI_flux_mw_bsn = df['OI_flux_mw'].copy()
OI_flux_mw_bsn[df['balmer_sn']<3] = df['OI_flux'][df['balmer_sn']<3]
df['OI_flux_mw_bsn'] = OI_flux_mw_bsn

In [29]:
# AGN flags

df['K03_flag'] = (df['K03_lauca'] == 'AGN') & (df['agn_sn'] > 3) & (-df['Ha_eqw'] > 3)
df['K01_flag'] = (df['K01_lauca'] == 'AGN') & (df['agn_sn'] > 3) & (-df['Ha_eqw'] > 3)

df['S06_flag_mw'] = (df['S06_lauca_mw'] == 'AGN') & (df['agn_sn_mw'] > 3) & (-df['Ha_eqw'] > 3)

df['K03_flag_mw_bsn'] = (df['K03_lauca_mw_bsn'] == 'AGN') & (df['agn_sn_mw_bsn'] > 3) & (-df['Ha_eqw'] > 3)
df['K01_flag_mw_bsn'] = (df['K01_lauca_mw_bsn'] == 'AGN') & (df['agn_sn_mw_bsn'] > 3) & (-df['Ha_eqw'] > 3)

#data['WISE_flag'] = 
#data['LERG_flag'] = 

In [30]:
# WHaN AGN

df['WHAN_flag'] = (df['Ha_eqw']<-3) & (np.log10(df['NII_flux']/df['Ha_flux']) > -0.4)

WHAN_flag_multi = []
for i in range(len(df['Ha_eqw'])):

    if (df['Ha_eqw'][i]>-3):

        WHAN_flag_multi.append('R')
        
    elif (df['Ha_eqw'][i]<-3) & (df['Ha_eqw'][i]>-6) & (np.log10(df['NII_flux'][i]/df['Ha_flux'][i]) > -0.4):

        WHAN_flag_multi.append('wAGN')

    elif (df['Ha_eqw'][i]<-6) & (np.log10(df['NII_flux'][i]/df['Ha_flux'][i]) > -0.4):

        WHAN_flag_multi.append('sAGN')

    elif (df['Ha_eqw'][i]<-3) & (np.log10(df['NII_flux'][i]/df['Ha_flux'][i]) < -0.4):

        WHAN_flag_multi.append('SF')

    else:

        print(df['Ha_eqw'][i], df['NII_flux'][i])
        WHAN_flag_multi.append('NaN')

df['WHAN_flag_multi'] = np.array(WHAN_flag_multi)

nan 0.0


/tmp/ipykernel_28198/4138628712.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (df['Ha_eqw'][i]>-3):
/tmp/ipykernel_28198/4138628712.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elif (df['Ha_eqw'][i]<-3) & (df['Ha_eqw'][i]>-6) & (np.log10(df['NII_flux'][i]/df['Ha_flux'][i]) > -0.4):
/tmp/ipykernel_28198/4138628712.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elif (df['Ha_eqw'][i]<-6) & (np.log10(df['NII_flux'

In [31]:
# WHaN AGN

df['WHAN_flag_mw_bsn'] = (df['Ha_eqw']<-3) & (np.log10(df['NII_flux_mw_bsn']/df['Ha_flux_mw_bsn']) > -0.4)

WHAN_flag_multi = []
for i in range(len(df['Ha_eqw'])):

    if (df['Ha_eqw'][i]>-3):

        WHAN_flag_multi.append('R')
        
    elif (df['Ha_eqw'][i]<-3) & (df['Ha_eqw'][i]>-6) & (np.log10(df['NII_flux_mw_bsn'][i]/df['Ha_flux_mw_bsn'][i]) > -0.4):

        WHAN_flag_multi.append('wAGN')

    elif (df['Ha_eqw'][i]<-6) & (np.log10(df['NII_flux_mw_bsn'][i]/df['Ha_flux_mw_bsn'][i]) > -0.4):

        WHAN_flag_multi.append('sAGN')

    elif (df['Ha_eqw'][i]<-3) & (np.log10(df['NII_flux_mw_bsn'][i]/df['Ha_flux_mw_bsn'][i]) < -0.4):

        WHAN_flag_multi.append('SF')

    else:

        print(df['Ha_eqw'][i], df['NII_flux_mw_bsn'][i])
        WHAN_flag_multi.append('NaN')

df['WHAN_flag_multi_mw_bsn'] = np.array(WHAN_flag_multi)

nan 0.0


/tmp/ipykernel_28198/4210732644.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (df['Ha_eqw'][i]>-3):
/tmp/ipykernel_28198/4210732644.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elif (df['Ha_eqw'][i]<-3) & (df['Ha_eqw'][i]>-6) & (np.log10(df['NII_flux_mw_bsn'][i]/df['Ha_flux_mw_bsn'][i]) > -0.4):
/tmp/ipykernel_28198/4210732644.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elif (df['Ha_eqw'][i]<-6) & (np.log10

In [32]:
import astropy.units as u
from astropy.cosmology import FlatLambdaCDM

cosmo = FlatLambdaCDM(H0=70, Om0=0.3)

d = cosmo.luminosity_distance(z = df['z'].to_numpy()).to(u.cm)

#                                    units from MPA/JHU catalogue page... (https://wwwmpa.mpa-garching.mpg.de/SDSS/DR7/raw_data.html)
df['L_OIII'] = df['OIII_flux'] * 1e-17 * 4 * np.pi * d**2 #erg/s
df['L_bol']  = df['OIII_flux'] * 1e-17 * 4 * np.pi * d**2 * 3500 #erg/s (conversion from Heckman et al. 2004)
df['L_bol_mw']  = df['OIII_flux_mw'] * 1e-17 * 4 * np.pi * d**2 * 3500 #erg/s (this time using mw corrected flux)
df['L_bol_mw_bsn']  = df['OIII_flux_mw_bsn'] * 1e-17 * 4 * np.pi * d**2 * 3500 #erg/s (this time using my hybrid mw-corrected fluxes)


kpc_per_arcsec = ((cosmo.arcsec_per_kpc_proper(z=df['z'].to_numpy()).value / 1e3) ** -1) / 1000

df['rpetro_kpc'] = df['rpetro']     * kpc_per_arcsec
df['res_kpc']    = df['resolution'] * kpc_per_arcsec
df['inner_kpc']  = 3 * kpc_per_arcsec

In [33]:
# adding my own columns

df['Nsigma_total'] = 10**df['MH2_total'] / 10**df['MH2_total_err']
df['Nsigma_inner'] = 10**df['MH2_inner'] / 10**df['MH2_inner_err']
df['Nsigma_outer'] = 10**df['MH2_outer'] / 10**df['MH2_outer_err']

df['fgas_total'] = 10**df['MH2_total'] / (10**df['MH2_total'] + 10**df['mstar_total'])
df['fgas_inner'] = 10**df['MH2_inner'] / (10**df['MH2_inner'] + 10**df['mstar_inner'])
df['fgas_outer'] = 10**df['MH2_outer'] / (10**df['MH2_outer'] + 10**df['mstar_outer'])

df['sfe_total'] = 10**df['MH2_total'] / 10**df['sfr_total']
df['sfe_inner'] = 10**df['MH2_inner'] / 10**df['sfr_inner']
df['sfe_outer'] = 10**df['MH2_outer'] / 10**df['sfr_outer']

df['grad_fgas'] = df['fgas_inner'] / df['fgas_outer']
df['grad_sfe'] = df['sfe_inner'] / df['sfe_outer']
df['grad_sfr'] = df['sfr_inner'] - df['sfr_outer']

In [34]:
kpc_per_arcsec = ((cosmo.arcsec_per_kpc_proper(z=df['z'].to_numpy()).value / 1e3) ** -1) / 1000

df['r_a_outer_kpc'] = df['r_a_outer'] * kpc_per_arcsec
df['r_b_outer_kpc'] = df['r_b_outer'] * kpc_per_arcsec
df['r_inner_kpc']  = 1.5 * kpc_per_arcsec

df['area_total_kpc2'] = np.pi * df['r_a_outer_kpc'] * df['r_b_outer_kpc']
df['area_inner_kpc2'] = np.pi * df['r_inner_kpc']**2
df['area_outer_kpc2'] = df['area_total_kpc2'] - df['area_inner_kpc2']

In [35]:
df['sigma_sfr_total'] = np.log10(10**df['sfr_total'] / df['area_total_kpc2'])
df['sigma_sfr_inner'] = np.log10(10**df['sfr_inner'] / df['area_inner_kpc2'])
df['sigma_sfr_outer'] = np.log10(10**df['sfr_outer'] / df['area_outer_kpc2'])

df['sigma_mstar_total'] = np.log10(10**df['mstar_total'] / df['area_total_kpc2'])
df['sigma_mstar_inner'] = np.log10(10**df['mstar_inner'] / df['area_inner_kpc2'])
df['sigma_mstar_outer'] = np.log10(10**df['mstar_outer'] / df['area_outer_kpc2'])

df['sigma_MH2_total'] = np.log10(10**df['MH2_total'] / df['area_total_kpc2'])
df['sigma_MH2_inner'] = np.log10(10**df['MH2_inner'] / df['area_inner_kpc2'])
df['sigma_MH2_outer'] = np.log10(10**df['MH2_outer'] / df['area_outer_kpc2'])

/opt/conda/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/conda/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/conda/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [36]:
L_OIII_corr_L09 = df['L_OIII'] * ((df['Ha_flux'] / df['Hb_flux']) / 3)**2.94 # Eq1 from Lamastra+09
L_OIII_corr_L09[df['balmer_sn']<2] = df['L_OIII'] # I don't trust the balmer decrement if we don't have a balmer_sn of at LEAST 2, so leave it uncorrected and consider it a lower limit
df['L_OIII_corr_L09_bsn'] = L_OIII_corr_L09 # add to dataframe

L_X_L09_bsn = 1e42 * 10**(1.11 + (1.02 * np.log10(df['L_OIII_corr_L09_bsn']/1e42))) # Eq3 from Lamastra+09
df['L_X_L09_bsn'] = L_X_L09_bsn # add to dataframe

/opt/conda/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [37]:
df

,source_name,project_code,year,MUID,AUID,ra,dec,z,rpetro,MRS,...,sigma_sfr_inner,sigma_sfr_outer,sigma_mstar_total,sigma_mstar_inner,sigma_mstar_outer,sigma_MH2_total,sigma_MH2_inner,sigma_MH2_outer,L_OIII_corr_L09_bsn,L_X_L09_bsn
objID,,,,,,,,,,,,,,,,,,,,,
587726033335550094,Abell,2011.0.00374.S,2011,uid://A002/X327408/X22f,uid://A002/X3b3400/X64c,210.258631,2.878469,0.251876,2.91803,10.534628,...,-0.223001,-3.187727,7.124471,9.109219,6.979364,6.128076,8.279227,5.867961,2.797003e+41,3.512584e+42
587741490891325518,587741490891325518_8241-3704,2018.1.00541.S,2018,uid://A001/X133d/Xe83,uid://A002/Xd74c3f/X24ad,126.568894,17.362436,0.065923,4.65495,21.589244,...,-0.318685,-3.879396,6.774690,9.098803,6.702272,5.971716,8.366052,5.885663,1.714312e+40,2.035971e+41
587722984441250023,HATLAS_J143336.7+005042,2017.1.00025.S,2017,uid://A001/X1284/X1534,uid://A002/Xca795f/X25d1,218.403317,0.845238,0.069836,1.43384,14.286183,...,-1.008691,-2.325777,8.236309,8.760187,8.037648,7.080184,8.043527,7.061985,8.244619e+39,9.649260e+40
588015508213989555,588015508213989555_8077-6104,2018.1.00541.S,2018,uid://A001/X133d/Xe93,uid://A002/Xd77a17/X1f36,42.032778,-0.752300,0.045791,6.22722,19.448789,...,-0.981319,-2.743803,7.328031,8.866642,7.266940,6.541642,8.043606,6.486942,3.219419e+39,3.697712e+40
587741421099155681,587741421099155681_8241-3703,2018.1.00541.S,2018,uid://A001/X133d/Xe7f,uid://A002/Xd74c3f/X273f,126.461162,18.166697,0.028862,4.86483,20.697749,...,-1.333461,-2.477165,7.260115,8.646967,7.179848,6.513773,7.702844,6.466344,1.157450e+39,1.302484e+40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
587729160054833498,AGC252580,2015.1.00405.S,2015,uid://A001/X2fe/X173,uid://A002/Xb08ef9/X104f,228.405346,4.322536,0.175406,3.60299,16.261699,...,-1.532308,-3.034385,7.110111,8.321314,7.008768,-inf,7.166708,-inf,3.797805e+39,4.376463e+40
587726102556180606,AGC242091,2015.1.00405.S,2015,uid://A001/X2fe/X17b,uid://A002/Xb07033/X845,211.344687,5.470736,0.195394,3.12167,16.272983,...,-1.007916,-2.618734,7.394916,8.521531,7.288604,6.482944,7.658400,6.343161,5.285582e+40,6.420291e+41
587727943492698312,AGC191728,2015.1.00405.S,2015,uid://A001/X2fe/X17f,uid://A002/Xb020f7/X1520,139.987509,1.647670,0.176348,2.02607,14.899537,...,-1.101940,-2.028581,7.847977,8.651360,7.678054,-inf,-inf,-inf,9.427018e+40,1.158409e+42


## Save as a science-ready dataframe

In [38]:
df.to_csv('salvage-scienceready-data-Oct17.csv')